In [9]:
import codecs
import os
import scipy as sp
import numpy as np
import re
import string
from string import punctuation
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem.snowball import SnowballStemmer
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import TruncatedSVD
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import libsvm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
C = 10  #懲罰常數
component = 800  #降維維度
most_simlr = 2  #擴展詞彙
max_fea = 1300 #特徵數量
vectorizer_count = CountVectorizer(analyzer='word',stop_words='english', encoding='utf-8',lowercase=True,max_features=max_fea)
svd = TruncatedSVD(n_components=component)

average_accuracy = 0
word2vec_average_accuracy = 0

#counter = int(input("請在此輸入要迭代的次數 : "))
counter = 5
def Accuracy_set_value(hit,miss):
    Accuracy = hit/(hit+miss)
    return Accuracy
#分析的評論句子用的前處理函數，會對輸入的句子去除標點符號，數字，非ascii字符，文字詞幹，轉換為寫小字母。
def sentence_preprocess(sentence):
    global punctuation
    punctuation += ","+"'"+"."+"&"+"!"+";"+":"+"..."+"´"+","
    ascii_list = string.ascii_letters+" "
    stemmer = SnowballStemmer("english")
    sentence = sentence.lower()
    sentence = list(sentence)
    out_sentence = "".join(stemmer.stem(c) for c in sentence if c not in punctuation)
    out_sentence= "".join([i for i in out_sentence if not i.isdigit()])
    out_sentence= "".join([i for i in out_sentence if i in ascii_list])
    out_sentence = " ".join(out_sentence.split())
    out_sentence2 = [stemmer.stem(plural) for plural in out_sentence.split()]
    out_sentence2 = " ".join(out_sentence2)
    if out_sentence2 == "":
        out_sentence2 = " The quick brown fox jumps over the lazy dog"
    out_sentence2 = out_sentence2.split()
    return out_sentence2  #回傳值是List型態
#Word2Vec使用的訓練語料庫必須為文字句子陣列，不須去詞幹，不須轉換大小寫。
def word2vec_train():
    global Word2Vec_Hotel_Traning_Model
    training_list = []
    with codecs.open("Booking_com_train_訓練Word2Vec語料.txt",encoding="utf-8", errors="backslashreplace") as sourceFile1:         
        training_file1 = sourceFile1.readlines()
        for document_1 in training_file1:
            training_list.append(document_1.split(" "))
        print("Booking_com_train_訓練Word2Vec語料.txt file1 converted,OK!!")
    Word2Vec_Hotel_Traning_Model = Word2Vec(training_list)
    print("Traning model is complete!!")
    print()
word2vec_train()
#接收的是一個句子的List  例如 [then,you,can,debug,the,threshold,with,generated ,tsv]
#用於把原本的句子按照Word2Vec相關性擴展成更大的句子清單
#基本上，一個原本有10個單字的句子，會被擴展成210個單字的句子
#擴展過後的200個單字會加進原本10個單字句子的後面。
#擴展的規則是，不去除重複的詞彙。
def word2vec_extend(sentence_word_list):
    if  type(sentence_word_list) == str:
        sentence_word_list = sentence_word_list.split()
    temp_list = []  #用於暫時存放擴展回傳的List
    for word_text in sentence_word_list:
        to_extend_word_list = Word2Vec_Hotel_Traning_Model.most_similar(word_text, topn=most_simlr)
        temp_sentence = ""
        for i in range(len(to_extend_word_list)):
            temp_sentence += to_extend_word_list[i][0]+" "
        temp_sentence = temp_sentence.split()
        temp_list += temp_sentence
    sentence_word_list += temp_list
    return sentence_word_list   #把擴展後的110單字句子List回傳
for Routine in range(counter):
    Routine +=1
    #實驗一 原本的評論語句不使用Word2Vec擴展
    #步驟0.讀入Train資料集 400筆
    Train_Booking_review = pd.read_csv("Booking_com_train_400筆.csv",sep=',',names=["Polarity","Context"])
    Train_Booking_review_Context = Train_Booking_review.Context
    Train_Booking_review_Polarity = Train_Booking_review.Polarity
    train_data_df_list = Train_Booking_review_Context.fillna(method="ffill")
    #步驟1.Train資料集 400筆 使用CountVectorizer 轉換為Bag of Word
    Train_corpus_data_features_count = vectorizer_count.fit_transform(train_data_df_list)
    Train_corpus_data_features_count_nd = Train_corpus_data_features_count.toarray()
    #Train_corpus_data_features_count_nd.shape
    #步驟2.提取出Train的Feature Word特徵詞串列List
    #Train_feature_list = vectorizer_count.get_feature_names()
    #print(Train_feature_list)
    #步驟3.讀入test資料集 100筆
    Test_Booking_review = pd.read_csv("Booking_com_test_100筆.csv",sep=',',names=["Polarity","Context"])
    Test_Booking_review_Context = Test_Booking_review.Context
    Test_Booking_review_Polarity = Test_Booking_review.Polarity
    Test_data_df_list = Test_Booking_review_Context.fillna(method="ffill")
    #print(Test_data_df_list)
    #步驟4.把test的資料集中的每一筆評論依序提取出來
    #步驟5.使用迭代的方式比對每一筆評論中的單字是否存在於Feature Word特徵詞串列List中
    #步驟6.以一則評論一個串列產生出Test的特徵詞串列-矩陣
    Test_corpus_data_features_count = vectorizer_count.transform(Test_data_df_list)
    Test_corpus_data_features_count_nd = Test_corpus_data_features_count.toarray()
    #Test_corpus_data_features_count_nd.shape
    #步驟7.把Train 與 Test 矩陣合併 意即 Train矩陣 後面串接 Test矩陣
    #合併後的矩陣命名為 「Combined_Matrix_01」
    #============去除SVD============#Combined_Matrix_01 = np.vstack((Train_corpus_data_features_count_nd,Test_corpus_data_features_count_nd))
    #print(Combined_Matrix_01.shape)
    #步驟8.把Combined_Matrix_01放入 Singular Value Decomposition函數中進行降維得到一個低維度的矩陣命名為「SVDed_Matrix_02」
    #============去除SVD============#Combined_Matrix_01 = svd.fit_transform(Combined_Matrix_01)
    #print(Combined_Matrix_01.shape)
    #步驟9.將SVDed_Matrix_02矩陣的Train範圍 與 Test範圍 再次切開
    #即矩陣的第0列到第399列為Train 命名為「Train_SVDed」
    #矩陣的第400列到第499列為Test 命名為「Test_SVDed」
    #============去除SVD============#Train_SVDed = Combined_Matrix_01[0:400,:]
    #============去除SVD============#Test_SVDed = Combined_Matrix_01[400:500,:]
    #print(Train_SVDed.shape)
    #print(Test_SVDed.shape)
    #步驟10.將Train_SVDed與相對應的Polarity Label送入Support Vector Machine 進行訓練得到模型
    Train_svc_model = svm.SVC(kernel='rbf', C=C,cache_size=2048).fit(X=Train_corpus_data_features_count_nd, y=Train_Booking_review_Polarity)
    #步驟11.將Test_SVDed送入訓練好的Support Vector Machine 模型取得預測結果
    Test_svc_predict = Train_svc_model.predict(Test_corpus_data_features_count_nd)
    #print(Test_svc_predict.shape)
    #步驟12.比對預測結果與正確答案計算出Accuracy值
    Hit = 0
    Miss = 0
    for index_number in range(len(Test_svc_predict)):
        if Test_svc_predict[index_number] == Test_Booking_review_Polarity[index_number]:
            Hit += 1
        else:
            Miss += 1
    No_Word2Vec_Acc = Accuracy_set_value(Hit,Miss)
    average_accuracy += No_Word2Vec_Acc
    print()
    print("迭代第 ",Routine," 次 未使用Word2Vec擴大的 Accuracy = ",No_Word2Vec_Acc*100,"%")
    #實驗二 使用Train資料集訓練Word2Vec Test用Word2Vec擴大然後測試。
    #步驟0.讀入Train資料集 400筆
    Train_Booking_review = pd.read_csv("Booking_com_train_400筆.csv",sep=',',names=["Polarity","Context"])
    Train_Booking_review_Context = Train_Booking_review.Context
    Train_Booking_review_Polarity = Train_Booking_review.Polarity
    train_data_df_list = Train_Booking_review_Context.fillna(method="ffill")
    #步驟1.將Train送入Word2Vec函數進行訓練得到Word2Vec模型稱為Word2Vec()。
    #步驟2.Train資料集 400筆 使用CountVectorizer 轉換為Bag of Word
    Train_corpus_data_features_count = vectorizer_count.fit_transform(train_data_df_list)
    Train_corpus_data_features_count_nd = Train_corpus_data_features_count.toarray()
    #print(Train_corpus_data_features_count_nd.shape)
    #步驟3.提取出Train的Feature Word特徵詞串列List
    #Train_feature_list = vectorizer_count.get_feature_names()
    #print(Train_feature_list)
    #步驟4.讀入test資料集 100筆
    Test_Booking_review = pd.read_csv("Booking_com_test_100筆.csv",sep=',',names=["Polarity","Context"])
    Test_Booking_review_Context = Test_Booking_review.Context
    Test_Booking_review_Polarity = Test_Booking_review.Polarity
    Test_data_df_list = Test_Booking_review_Context.fillna(method="ffill")
    #print(Test_data_df_list)
    #步驟5.把Test的資料集送入Word2Vec()模型進行擴大，得到的新資料集命名為 Test_Word2Veced
    train_data_df_list = []
    #print("progress",end="")
    for to_extend in Test_data_df_list.tolist():
        try:
            train_data_df_list.append(" ".join(word2vec_extend(to_extend)))
            #print(".",end="")
        except KeyError:
            #print("*",end="")
            train_data_df_list.append(to_extend)
            pass
        except TypeError:
            #print("*",end="")
            train_data_df_list.append(to_extend)
            pass
    #print("Done ...")
    #步驟6.把Test_Word2Veced的資料集中的每一筆評論依序提取出來
    #步驟7.使用迭代的方式比對每一筆評論中的單字是否存在於Feature Word特徵詞串列List中
    #如果存在者就按照原本的次數進行累加，原本不存在者就從1次開始累加。
    #如果不在Feature Word特徵詞串列List的詞彙就丟棄
    #(意思就是，重複出現在辭彙保留，原本沒有的字因為經由擴大才出現的辭彙則加入Feature Matrix)
    #步驟8.以一則評論一個串列產生出Test_Word2Veced的特徵詞串列-矩陣
    Test_corpus_data_features_count = vectorizer_count.transform(train_data_df_list)
    Test_corpus_data_features_count_nd = Test_corpus_data_features_count.toarray()
    #print(Test_corpus_data_features_count_nd.shape)
    #步驟9.把 Train 與 Test_Word2Veced 矩陣合併 意即 Train 矩陣 後面串接 Test 矩陣合併後的矩陣命名為 「Combined_Matrix_01」
    #Combined_Matrix_01 = np.vstack((Train_corpus_data_features_count_nd,Test_corpus_data_features_count_nd))
    #print(Combined_Matrix_01.shape)
    #步驟10.把Combined_Matrix_01放入 Singular Value Decomposition函數中進行降維得到一個低維度的矩陣命名為「SVDed_Matrix_02」
    #============去除SVD============#Combined_Matrix_01 = svd.fit_transform(Combined_Matrix_01)
    #print(Combined_Matrix_01.shape)
    #步驟11.將SVDed_Matrix_02矩陣的Train範圍 與 Test範圍 再次切開
    #即矩陣的第0列到第399列為Train 命名為「Train_SVDed」
    #矩陣的第400列到第499列為Test 命名為「Test_Word2Veced_SVDed」
    #============去除SVD============#Train_SVDed = Combined_Matrix_01[0:400,:]
    #============去除SVD============#Test_Word2Veced_SVDed = Combined_Matrix_01[400:500,:]
    #print(Train_SVDed.shape)
    #print(Test_Word2Veced_SVDed.shape)
    #步驟12.將Train_SVDed與相對應的Polarity Label送入Support Vector Machine 進行訓練得到模型
    Train_svc_model = svm.SVC(kernel='rbf', C=C,cache_size=2048).fit(X=Train_corpus_data_features_count_nd, y=Train_Booking_review_Polarity)
    #步驟13.將Test_SVDed送入訓練好的Support Vector Machine 模型取得預測結果
    Test_svc_predict = Train_svc_model.predict(Test_corpus_data_features_count_nd)
    #print(Test_svc_predict.shape)
    #步驟14.比對預測結果與正確答案計算出Accuracy值
    Hit = 0
    Miss = 0
    for index_number in range(len(Test_svc_predict)):
        if Test_svc_predict[index_number] == Test_Booking_review_Polarity[index_number]:  #錯誤在這  原本寫成Train_Booking_review_Polarity
            Hit += 1
        else:
            Miss += 1
    Yes_Word2Vec_Acc = Accuracy_set_value(Hit,Miss)
    word2vec_average_accuracy += Yes_Word2Vec_Acc
    print("迭代第 ",Routine," 次  有使用Word2Vec擴大的 Accuracy = ",Yes_Word2Vec_Acc*100,"%")
print()
print("未使用Word2Vec擴大的平均 Accuracy =",((average_accuracy)/counter)*100,"%")    
print("有使用Word2Vec擴大的平均 Accuracy =",((word2vec_average_accuracy)/counter)*100,"%")

請在此輸入要迭代的次數 : 5
Booking_com_train_訓練Word2Vec語料.txt file1 converted,OK!!
Traning model is complete!!


迭代第  1  次 未使用Word2Vec擴大的 Accuracy =  78.0 %
迭代第  1  次  有使用Word2Vec擴大的 Accuracy =  78.0 %

迭代第  2  次 未使用Word2Vec擴大的 Accuracy =  78.0 %
迭代第  2  次  有使用Word2Vec擴大的 Accuracy =  78.0 %

迭代第  3  次 未使用Word2Vec擴大的 Accuracy =  78.0 %
迭代第  3  次  有使用Word2Vec擴大的 Accuracy =  78.0 %

迭代第  4  次 未使用Word2Vec擴大的 Accuracy =  78.0 %
迭代第  4  次  有使用Word2Vec擴大的 Accuracy =  78.0 %

迭代第  5  次 未使用Word2Vec擴大的 Accuracy =  78.0 %
迭代第  5  次  有使用Word2Vec擴大的 Accuracy =  78.0 %

未使用Word2Vec擴大的平均 Accuracy = 78.0 %
有使用Word2Vec擴大的平均 Accuracy = 78.0 %


In [12]:
import codecs
import os
import scipy as sp
import numpy as np
import re
import string
from string import punctuation
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem.snowball import SnowballStemmer
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import TruncatedSVD
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import libsvm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn import grid_search


parameters ={'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}
#C = 10  #懲罰常數
component = 1000  #降維維度
most_simlr = 10  #擴展詞彙
max_fea = 2000 #特徵數量
vectorizer_count = CountVectorizer(analyzer='word',stop_words='english', encoding='utf-8',lowercase=True,max_features=max_fea)
svd = TruncatedSVD(n_components=component)

average_accuracy = 0
word2vec_average_accuracy = 0

#counter = int(input("請在此輸入要迭代的次數 : "))
counter = 5
def Accuracy_set_value(hit,miss):
    Accuracy = hit/(hit+miss)
    return Accuracy
#分析的評論句子用的前處理函數，會對輸入的句子去除標點符號，數字，非ascii字符，文字詞幹，轉換為寫小字母。
def sentence_preprocess(sentence):
    global punctuation
    punctuation += ","+"'"+"."+"&"+"!"+";"+":"+"..."+"´"+","
    ascii_list = string.ascii_letters+" "
    stemmer = SnowballStemmer("english")
    sentence = sentence.lower()
    sentence = list(sentence)
    out_sentence = "".join(stemmer.stem(c) for c in sentence if c not in punctuation)
    out_sentence= "".join([i for i in out_sentence if not i.isdigit()])
    out_sentence= "".join([i for i in out_sentence if i in ascii_list])
    out_sentence = " ".join(out_sentence.split())
    out_sentence2 = [stemmer.stem(plural) for plural in out_sentence.split()]
    out_sentence2 = " ".join(out_sentence2)
    if out_sentence2 == "":
        out_sentence2 = " The quick brown fox jumps over the lazy dog"
    out_sentence2 = out_sentence2.split()
    return out_sentence2  #回傳值是List型態
#Word2Vec使用的訓練語料庫必須為文字句子陣列，不須去詞幹，不須轉換大小寫。
def word2vec_train():
    global Word2Vec_Hotel_Traning_Model
    training_list = []
    with codecs.open("Booking_com_train_訓練Word2Vec語料.txt",encoding="utf-8", errors="backslashreplace") as sourceFile1:         
        training_file1 = sourceFile1.readlines()
        for document_1 in training_file1:
            training_list.append(document_1.split(" "))
        print("Booking_com_train_訓練Word2Vec語料.txt file1 converted,OK!!")
    Word2Vec_Hotel_Traning_Model = Word2Vec(training_list)
    print("Traning model is complete!!")
    print()
word2vec_train()
#接收的是一個句子的List  例如 [then,you,can,debug,the,threshold,with,generated ,tsv]
#用於把原本的句子按照Word2Vec相關性擴展成更大的句子清單
#基本上，一個原本有10個單字的句子，會被擴展成210個單字的句子
#擴展過後的200個單字會加進原本10個單字句子的後面。
#擴展的規則是，不去除重複的詞彙。
def word2vec_extend(sentence_word_list):
    if  type(sentence_word_list) == str:
        sentence_word_list = sentence_word_list.split()
    temp_list = []  #用於暫時存放擴展回傳的List
    for word_text in sentence_word_list:
        to_extend_word_list = Word2Vec_Hotel_Traning_Model.most_similar(word_text, topn=most_simlr)
        temp_sentence = ""
        for i in range(len(to_extend_word_list)):
            temp_sentence += to_extend_word_list[i][0]+" "
        temp_sentence = temp_sentence.split()
        temp_list += temp_sentence
    sentence_word_list += temp_list
    return sentence_word_list   #把擴展後的110單字句子List回傳
for Routine in range(counter):
    Routine +=1
    #實驗一 原本的評論語句不使用Word2Vec擴展
    #步驟0.讀入Train資料集 400筆
    Train_Booking_review = pd.read_csv("Booking_com_train_400筆.csv",sep=',',names=["Polarity","Context"])
    Train_Booking_review_Context = Train_Booking_review.Context
    Train_Booking_review_Polarity = Train_Booking_review.Polarity
    train_data_df_list = Train_Booking_review_Context.fillna(method="ffill")
    #步驟1.Train資料集 400筆 使用CountVectorizer 轉換為Bag of Word
    Train_corpus_data_features_count = vectorizer_count.fit_transform(train_data_df_list)
    Train_corpus_data_features_count_nd = Train_corpus_data_features_count.toarray()
    #Train_corpus_data_features_count_nd.shape
    #步驟2.提取出Train的Feature Word特徵詞串列List
    #Train_feature_list = vectorizer_count.get_feature_names()
    #print(Train_feature_list)
    #步驟3.讀入test資料集 100筆
    Test_Booking_review = pd.read_csv("Booking_com_test_100筆.csv",sep=',',names=["Polarity","Context"])
    Test_Booking_review_Context = Test_Booking_review.Context
    Test_Booking_review_Polarity = Test_Booking_review.Polarity
    Test_data_df_list = Test_Booking_review_Context.fillna(method="ffill")
    #print(Test_data_df_list)
    #步驟4.把test的資料集中的每一筆評論依序提取出來
    #步驟5.使用迭代的方式比對每一筆評論中的單字是否存在於Feature Word特徵詞串列List中
    #步驟6.以一則評論一個串列產生出Test的特徵詞串列-矩陣
    Test_corpus_data_features_count = vectorizer_count.transform(Test_data_df_list)
    Test_corpus_data_features_count_nd = Test_corpus_data_features_count.toarray()
    #Test_corpus_data_features_count_nd.shape
    #步驟7.把Train 與 Test 矩陣合併 意即 Train矩陣 後面串接 Test矩陣
    #合併後的矩陣命名為 「Combined_Matrix_01」
    #============去除SVD============#Combined_Matrix_01 = np.vstack((Train_corpus_data_features_count_nd,Test_corpus_data_features_count_nd))
    #print(Combined_Matrix_01.shape)
    #步驟8.把Combined_Matrix_01放入 Singular Value Decomposition函數中進行降維得到一個低維度的矩陣命名為「SVDed_Matrix_02」
    #============去除SVD============#Combined_Matrix_01 = svd.fit_transform(Combined_Matrix_01)
    #print(Combined_Matrix_01.shape)
    #步驟9.將SVDed_Matrix_02矩陣的Train範圍 與 Test範圍 再次切開
    #即矩陣的第0列到第399列為Train 命名為「Train_SVDed」
    #矩陣的第400列到第499列為Test 命名為「Test_SVDed」
    #============去除SVD============#Train_SVDed = Combined_Matrix_01[0:400,:]
    #============去除SVD============#Test_SVDed = Combined_Matrix_01[400:500,:]
    #print(Train_SVDed.shape)
    #print(Test_SVDed.shape)
    #步驟10.將Train_SVDed與相對應的Polarity Label送入Support Vector Machine 進行訓練得到模型
    Train_svc_model = grid_search.GridSearchCV(SVC(kernel='rbf',cache_size=2048), parameters).fit(X=Train_corpus_data_features_count_nd, y=Train_Booking_review_Polarity)
    #步驟11.將Test_SVDed送入訓練好的Support Vector Machine 模型取得預測結果
    Test_svc_predict = Train_svc_model.predict(Test_corpus_data_features_count_nd)
    #print(Test_svc_predict.shape)
    #步驟12.比對預測結果與正確答案計算出Accuracy值
    Hit = 0
    Miss = 0
    for index_number in range(len(Test_svc_predict)):
        if Test_svc_predict[index_number] == Test_Booking_review_Polarity[index_number]:
            Hit += 1
        else:
            Miss += 1
    No_Word2Vec_Acc = Accuracy_set_value(Hit,Miss)
    average_accuracy += No_Word2Vec_Acc
    print()
    print("迭代第 ",Routine," 次 未使用Word2Vec擴大的 Accuracy = ",No_Word2Vec_Acc*100,"%")
    #實驗二 使用Train資料集訓練Word2Vec Test用Word2Vec擴大然後測試。
    #步驟0.讀入Train資料集 400筆
    Train_Booking_review = pd.read_csv("Booking_com_train_400筆.csv",sep=',',names=["Polarity","Context"])
    Train_Booking_review_Context = Train_Booking_review.Context
    Train_Booking_review_Polarity = Train_Booking_review.Polarity
    train_data_df_list = Train_Booking_review_Context.fillna(method="ffill")
    #步驟1.將Train送入Word2Vec函數進行訓練得到Word2Vec模型稱為Word2Vec()。
    #步驟2.Train資料集 400筆 使用CountVectorizer 轉換為Bag of Word
    Train_corpus_data_features_count = vectorizer_count.fit_transform(train_data_df_list)
    Train_corpus_data_features_count_nd = Train_corpus_data_features_count.toarray()
    #print(Train_corpus_data_features_count_nd.shape)
    #步驟3.提取出Train的Feature Word特徵詞串列List
    #Train_feature_list = vectorizer_count.get_feature_names()
    #print(Train_feature_list)
    #步驟4.讀入test資料集 100筆
    Test_Booking_review = pd.read_csv("Booking_com_test_100筆.csv",sep=',',names=["Polarity","Context"])
    Test_Booking_review_Context = Test_Booking_review.Context
    Test_Booking_review_Polarity = Test_Booking_review.Polarity
    Test_data_df_list = Test_Booking_review_Context.fillna(method="ffill")
    #print(Test_data_df_list)
    #步驟5.把Test的資料集送入Word2Vec()模型進行擴大，得到的新資料集命名為 Test_Word2Veced
    train_data_df_list = []
    #print("progress",end="")
    for to_extend in Test_data_df_list.tolist():
        try:
            train_data_df_list.append(" ".join(word2vec_extend(to_extend)))
            #print(".",end="")
        except KeyError:
            #print("*",end="")
            train_data_df_list.append(to_extend)
            pass
        except TypeError:
            #print("*",end="")
            train_data_df_list.append(to_extend)
            pass
    #print("Done ...")
    #步驟6.把Test_Word2Veced的資料集中的每一筆評論依序提取出來
    #步驟7.使用迭代的方式比對每一筆評論中的單字是否存在於Feature Word特徵詞串列List中
    #如果存在者就按照原本的次數進行累加，原本不存在者就從1次開始累加。
    #如果不在Feature Word特徵詞串列List的詞彙就丟棄
    #(意思就是，重複出現在辭彙保留，原本沒有的字因為經由擴大才出現的辭彙則加入Feature Matrix)
    #步驟8.以一則評論一個串列產生出Test_Word2Veced的特徵詞串列-矩陣
    Test_corpus_data_features_count = vectorizer_count.transform(train_data_df_list)
    Test_corpus_data_features_count_nd = Test_corpus_data_features_count.toarray()
    #print(Test_corpus_data_features_count_nd.shape)
    #步驟9.把 Train 與 Test_Word2Veced 矩陣合併 意即 Train 矩陣 後面串接 Test 矩陣合併後的矩陣命名為 「Combined_Matrix_01」
    #Combined_Matrix_01 = np.vstack((Train_corpus_data_features_count_nd,Test_corpus_data_features_count_nd))
    #print(Combined_Matrix_01.shape)
    #步驟10.把Combined_Matrix_01放入 Singular Value Decomposition函數中進行降維得到一個低維度的矩陣命名為「SVDed_Matrix_02」
    #============去除SVD============#Combined_Matrix_01 = svd.fit_transform(Combined_Matrix_01)
    #print(Combined_Matrix_01.shape)
    #步驟11.將SVDed_Matrix_02矩陣的Train範圍 與 Test範圍 再次切開
    #即矩陣的第0列到第399列為Train 命名為「Train_SVDed」
    #矩陣的第400列到第499列為Test 命名為「Test_Word2Veced_SVDed」
    #============去除SVD============#Train_SVDed = Combined_Matrix_01[0:400,:]
    #============去除SVD============#Test_Word2Veced_SVDed = Combined_Matrix_01[400:500,:]
    #print(Train_SVDed.shape)
    #print(Test_Word2Veced_SVDed.shape)
    #步驟12.將Train_SVDed與相對應的Polarity Label送入Support Vector Machine 進行訓練得到模型
    Train_svc_model = grid_search.GridSearchCV(SVC(kernel='rbf',cache_size=2048), parameters).fit(X=Train_corpus_data_features_count_nd, y=Train_Booking_review_Polarity)
    #步驟13.將Test_SVDed送入訓練好的Support Vector Machine 模型取得預測結果
    Test_svc_predict = Train_svc_model.predict(Test_corpus_data_features_count_nd)
    #print(Test_svc_predict.shape)
    #步驟14.比對預測結果與正確答案計算出Accuracy值
    Hit = 0
    Miss = 0
    for index_number in range(len(Test_svc_predict)):
        if Test_svc_predict[index_number] == Test_Booking_review_Polarity[index_number]:  #錯誤在這  原本寫成Train_Booking_review_Polarity
            Hit += 1
        else:
            Miss += 1
    Yes_Word2Vec_Acc = Accuracy_set_value(Hit,Miss)
    word2vec_average_accuracy += Yes_Word2Vec_Acc
    print("迭代第 ",Routine," 次  有使用Word2Vec擴大的 Accuracy = ",Yes_Word2Vec_Acc*100,"%")
print()
print("未使用Word2Vec擴大的平均 Accuracy =",((average_accuracy)/counter)*100,"%")    
print("有使用Word2Vec擴大的平均 Accuracy =",((word2vec_average_accuracy)/counter)*100,"%")

Booking_com_train_訓練Word2Vec語料.txt file1 converted,OK!!
Traning model is complete!!


迭代第  1  次 未使用Word2Vec擴大的 Accuracy =  82.0 %
迭代第  1  次  有使用Word2Vec擴大的 Accuracy =  80.0 %

迭代第  2  次 未使用Word2Vec擴大的 Accuracy =  82.0 %
迭代第  2  次  有使用Word2Vec擴大的 Accuracy =  80.0 %

迭代第  3  次 未使用Word2Vec擴大的 Accuracy =  82.0 %
迭代第  3  次  有使用Word2Vec擴大的 Accuracy =  80.0 %

迭代第  4  次 未使用Word2Vec擴大的 Accuracy =  82.0 %
迭代第  4  次  有使用Word2Vec擴大的 Accuracy =  80.0 %

迭代第  5  次 未使用Word2Vec擴大的 Accuracy =  82.0 %
迭代第  5  次  有使用Word2Vec擴大的 Accuracy =  80.0 %

未使用Word2Vec擴大的平均 Accuracy = 82.0 %
有使用Word2Vec擴大的平均 Accuracy = 80.0 %
